## Backward difference coding Using `Categorical encoders`

https://contrib.scikit-learn.org/categorical-encoding/backward_difference.html#

### Installing categorical encoders

In [2]:
! pip install category-encoders

     |████████████████████████████████| 92kB 1.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols

import category_encoders as ce

### Creating a dataframe
The same 'ol Iris dataset

In [4]:
iris_data = pd.read_csv("datasets/iris.csv")

iris_data.sample(10)

,Species,sepal_length,sepal_width,petal_length,petal_width
59,Iris-versicolor,5.2,2.7,3.9,1.4
72,Iris-versicolor,6.3,2.5,4.9,1.5
140,Iris-virginica,6.7,3.1,5.6,2.4
29,Iris-setosa,4.7,3.2,1.6,0.2
105,Iris-virginica,7.6,3.0,6.6,2.1
41,Iris-setosa,4.5,2.3,1.3,0.3
55,Iris-versicolor,5.7,2.8,4.5,1.3
143,Iris-virginica,6.8,3.2,5.9,2.3
62,Iris-versicolor,6.0,2.2,4.0,1.0
48,Iris-setosa,5.3,3.7,1.5,0.2


In [5]:
iris_data.drop(columns=['sepal_length', 'sepal_width', 'petal_width'], 
               inplace=True)

iris_data.sample(5)

,Species,petal_length
122,Iris-virginica,6.7
81,Iris-versicolor,3.7
78,Iris-versicolor,4.5
88,Iris-versicolor,4.1
131,Iris-virginica,6.4


In [6]:
iris_data.describe()

,petal_length
count,150.000000
mean,3.758667
std,1.764420
min,1.000000
25%,1.600000
50%,4.350000
75%,5.100000
max,6.900000


In [7]:
iris_data.groupby(by='Species').mean()

,petal_length
Species,
Iris-setosa,1.464
Iris-versicolor,4.260
Iris-virginica,5.552


In [8]:
iris_data.groupby(by='Species').mean().diff()

,petal_length
Species,
Iris-setosa,NaN
Iris-versicolor,2.796
Iris-virginica,1.292


In [10]:
mod = ols("petal_length ~ C(Species, Diff)", 
          data=iris_data)

res = mod.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           petal_length   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     1179.
Date:                Mon, 01 Jul 2019   Prob (F-statistic):           3.05e-91
Time:                        08:18:32   Log-Likelihood:                -84.840
No. Observations:                 150   AIC:                             175.7
Df Residuals:                     147   BIC:                             184.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               3.7587      0.035    106.978      0.000       3.689       3.828
C(Species, Diff)[D.Iris-setosa]         2.7960      0.086     32.488      0.000       2.626       2.966
C(Species, Diff)[D.Iris-versicolor]     1.2920      0.086     15.012      0.000       1.122       1.462
==============================================================================
Omnibus:                        4.393   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                5.370
Skew:                           0.121   Prob(JB):                       0.0682
Kurtosis:                       3.895   Cond. No.                         3.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

https://contrib.scikit-learn.org/categorical-encoding/backward_difference.html

### Creating a object and giving the column which need to be encoded as argument

In [14]:
encoder = ce.BackwardDifferenceEncoder(cols=['Species'])
encoder

BackwardDifferenceEncoder(cols=['Species'], drop_invariant=False,
                          handle_missing='value', handle_unknown='value',
                          mapping=None, return_df=True, verbose=0)

### Dataframe with Encoded columns of  `color`

In [18]:
species_encoded = encoder.fit_transform(iris_data)

species_encoded.head()

,intercept,Species_0,Species_1,petal_length
0,1,-0.666667,-0.333333,1.4
1,1,-0.666667,-0.333333,1.4
2,1,-0.666667,-0.333333,1.3
3,1,-0.666667,-0.333333,1.5
4,1,-0.666667,-0.333333,1.4


In [19]:
species_encoded.sample(5)

,intercept,Species_0,Species_1,petal_length
144,1,0.333333,0.666667,5.7
91,1,0.333333,-0.333333,4.6
50,1,0.333333,-0.333333,4.7
98,1,0.333333,-0.333333,3.0
12,1,-0.666667,-0.333333,1.4


In [20]:
encoded_iris = pd.concat([iris_data['Species'], species_encoded], 
                         axis = 1)

encoded_iris.sample(10)

,Species,intercept,Species_0,Species_1,petal_length
33,Iris-setosa,1,-0.666667,-0.333333,1.4
15,Iris-setosa,1,-0.666667,-0.333333,1.5
75,Iris-versicolor,1,0.333333,-0.333333,4.4
124,Iris-virginica,1,0.333333,0.666667,5.7
102,Iris-virginica,1,0.333333,0.666667,5.9
14,Iris-setosa,1,-0.666667,-0.333333,1.2
69,Iris-versicolor,1,0.333333,-0.333333,3.9
135,Iris-virginica,1,0.333333,0.666667,6.1
73,Iris-versicolor,1,0.333333,-0.333333,4.7
121,Iris-virginica,1,0.333333,0.666667,4.9


In [21]:
X = encoded_iris.drop(columns=['Species', 'petal_length'])

y = encoded_iris.petal_length

In [22]:
X.head()

,intercept,Species_0,Species_1
0,1,-0.666667,-0.333333
1,1,-0.666667,-0.333333
2,1,-0.666667,-0.333333
3,1,-0.666667,-0.333333
4,1,-0.666667,-0.333333


In [23]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept=False)

linear_model.fit(X, y)

print("Training_score : " , linear_model.score(X, y))

Training_score :  0.9413189735606261


In [24]:
linear_model.coef_

array([3.75866667, 2.796     , 1.292     ])

In [25]:
linear_model.intercept_

0.0

## Helmert Coding using `Categorical encoders`

### Importing modules

In [15]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols

### Loading the dataset
https://www.kaggle.com/toramky/automobile-dataset

In [2]:
car_data = pd.read_csv('datasets/auto-mpg.csv', na_values='?')

car_data.sample(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
61,21.0,4,122.0,86.0,2226,16.5,72,1,ford pinto runabout
46,22.0,4,140.0,72.0,2408,19.0,71,1,chevrolet vega (sw)
353,33.0,4,105.0,74.0,2190,14.2,81,2,volkswagen jetta
26,10.0,8,307.0,200.0,4376,15.0,70,1,chevy c20
208,13.0,8,318.0,150.0,3940,13.2,76,1,plymouth volare premier v8
148,26.0,4,116.0,75.0,2246,14.0,74,2,fiat 124 tc
27,11.0,8,318.0,210.0,4382,13.5,70,1,dodge d200
234,24.5,4,151.0,88.0,2740,16.0,77,1,pontiac sunbird coupe
31,25.0,4,113.0,95.0,2228,14.0,71,3,toyota corona
41,14.0,8,318.0,150.0,4096,13.0,71,1,plymouth fury iii


In [3]:
car_data = car_data[['mpg', 'cylinders']]

In [4]:
car_data.dropna(inplace=True)

In [5]:
car_data.shape

(398, 2)

In [6]:
car_data.sample(10)

,mpg,cylinders
318,29.8,4
212,16.5,8
33,19.0,6
160,17.0,6
126,21.0,6
163,18.0,6
104,12.0,8
343,39.1,4
386,25.0,6
233,29.0,4


### Printing the different classes present

In [7]:
car_data['cylinders'].unique()

array([8, 4, 6, 3, 5])

In [9]:
car_data.sort_values(by=['cylinders'], inplace=True)

car_data.reset_index(inplace=True, drop=True)

car_data.head(10)

,mpg,cylinders
0,18.0,3
1,19.0,3
2,23.7,3
3,21.5,3
4,32.3,4
5,35.1,4
6,39.0,4
7,39.1,4
8,30.0,4
9,25.8,4


In [16]:
car_data.mean()

mpg          23.514573
cylinders     5.454774
dtype: float64

In [37]:
car_data_grouped = car_data.groupby(by=['cylinders']).mean()

car_data_grouped

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [38]:
car_data_grouped['mpg'].mean()

22.43045049087596

In [46]:
coefficient_cylinder_4 = \
    (car_data_grouped.loc[4]['mpg'] - car_data_grouped.loc[3]['mpg']) / 2

coefficient_cylinder_4

4.368382352941174

In [47]:
mean_34 = (car_data_grouped.loc[3]['mpg'] + car_data_grouped.loc[4]['mpg']) / 2

coefficient_cylinder_5 = (car_data_grouped.loc[5]['mpg'] - mean_34) / 3

coefficient_cylinder_5

0.8160947712418304

In [48]:
mod = ols("mpg ~ C(cylinders, Helmert)", 
          data=car_data)

res = mod.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.637
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     172.6
Date:                Mon, 01 Jul 2019   Prob (F-statistic):           3.68e-85
Time:                        10:14:18   Log-Likelihood:                -1180.8
No. Observations:                 398   AIC:                             2372.
Df Residuals:                     393   BIC:                             2392.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     22.4305      0.739     30.353      0.000      20.978      23.883
C(cylinders, Helmert)[H.4]     4.3684      1.194      3.657      0.000       2.020       6.717
C(cylinders, Helmert)[H.5]     0.8161      0.994      0.821      0.412      -1.138       2.770
C(cylinders, Helmert)[H.6]    -1.4372      0.329     -4.371      0.000      -2.084      -0.791
C(cylinders, Helmert)[H.8]    -1.8668      0.206     -9.079      0.000      -2.271      -1.463
==============================================================================
Omnibus:                       48.011   Durbin-Watson:                   1.464
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.510
Skew:                           0.793   Prob(JB):                     2.96e-16
Kurtosis:                       4.341   Cond. No.                         12.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Importing categorical encoders

In [49]:
import category_encoders as ce

### Creating a HelmertEncoder on the 'cylinders' column

In [62]:
ce_helmert = ce.HelmertEncoder(cols = ['cylinders'])
ce_helmert

HelmertEncoder(cols=['cylinders'], drop_invariant=False,
               handle_missing='indicator', handle_unknown='indicator',
               mapping=None, return_df=True, verbose=0)

### Encoding the dataset

In [51]:
car_he = ce_helmert.fit_transform(car_data)
car_he.sample(10)

,mpg,cylinders_0,cylinders_1,cylinders_2,cylinders_3
208,25.4,0.0,2.0,-1.0,-1.0
331,13.0,0.0,0.0,0.0,4.0
111,23.8,1.0,-1.0,-1.0,-1.0
241,18.0,0.0,0.0,3.0,-1.0
62,27.0,1.0,-1.0,-1.0,-1.0
143,34.1,1.0,-1.0,-1.0,-1.0
232,20.0,0.0,0.0,3.0,-1.0
127,24.0,1.0,-1.0,-1.0,-1.0
150,31.8,1.0,-1.0,-1.0,-1.0
243,25.4,0.0,0.0,3.0,-1.0


In [52]:
pd.concat([car_data['cylinders'], car_he], axis=1).sample(10)

,cylinders,mpg,cylinders_0,cylinders_1,cylinders_2,cylinders_3
79,4,24.0,1.0,-1.0,-1.0,-1.0
278,6,16.0,0.0,0.0,3.0,-1.0
337,8,18.2,0.0,0.0,0.0,4.0
252,6,20.2,0.0,0.0,3.0,-1.0
13,4,23.6,1.0,-1.0,-1.0,-1.0
281,6,20.0,0.0,0.0,3.0,-1.0
31,4,29.9,1.0,-1.0,-1.0,-1.0
374,8,16.0,0.0,0.0,0.0,4.0
30,4,38.0,1.0,-1.0,-1.0,-1.0
376,8,13.0,0.0,0.0,0.0,4.0


In [54]:
X = car_he.drop(columns = ['mpg'], axis=1)

y = car_he['mpg']

X.sample(10)

,cylinders_0,cylinders_1,cylinders_2,cylinders_3
47,1.0,-1.0,-1.0,-1.0
370,0.0,0.0,0.0,4.0
77,1.0,-1.0,-1.0,-1.0
292,0.0,0.0,3.0,-1.0
205,1.0,-1.0,-1.0,-1.0
163,1.0,-1.0,-1.0,-1.0
254,0.0,0.0,3.0,-1.0
382,0.0,0.0,0.0,4.0
60,1.0,-1.0,-1.0,-1.0
227,0.0,0.0,3.0,-1.0


In [55]:
y.head(10)

0    18.0
1    19.0
2    23.7
3    21.5
4    32.3
5    35.1
6    39.0
7    39.1
8    30.0
9    25.8
Name: mpg, dtype: float64

In [60]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

linear_model.fit(X, y)

print("Training_score : " , linear_model.score(X, y))

Training_score :  0.6372420899156167


In [61]:
linear_model.coef_

array([ 4.36838235,  0.81609477, -1.43719071, -1.86683592])